In [ ]:
junction_id = {'KLONGTEI': 'cluster_1892287670_272491964_272492178',
      'RAMA4': 'cluster_272488163_282390730_66263210_66263222',
      'NARANONG': 'cluster_272488164_272492179_3457051443_61907354',
      'SUNLAKAKHON': 'gneJ83',
      'KASEMRAT': 'cluster_272448137_272555800_272555808_7660045934_7710268409',
      'ATTHAKAWI_RAMA4' : '270329335'}
junction_name = list(junction_id.keys())

In [2]:
# import gym
# from gym import error, spaces
import gym
import csv
import os, sys
sys.path.append(os.path.join('/home/ring/sumo-svn/', 'tools'))
import traci
import traci.constants as tc
import numpy as np
from sumolib import checkBinary
import datetime
import matplotlib
import matplotlib.pyplot as plt
import torch
import copy
import random
import pandas as pd
from multiprocessing.pool import ThreadPool
from itertools import zip_longest

In [3]:
namelane_csv = pd.read_csv('namelane_KASEMRAT.csv')
namelane_df = pd.DataFrame(namelane_csv, columns = ['name' , 'id'])
NAME = namelane_df.set_index('name')
ID = namelane_df.set_index('id')
if NAME.loc['KASEMRAT_EB_0_0_XSXX','id'] == '459551209#3_0':
    print('ok')

ok


In [4]:
NAME.loc[:,'id']

name
KASEMRAT_EB_0_0_XSXX                      459551209#3_0
KASEMRAT_EB_0_1_XSXX                      459551209#3_1
KASEMRAT_EB_0_2_XSXX                      459551209#3_2
KASEMRAT_EB_0_3_XSRT                      459551209#3_3
KASEMRAT_EB_1_0_LSXX                      459551209#0_0
                                              ...      
MASUKGRIDLOCK_ATTHAKAWI_SB_2_1_LXXX        27702347#6_1
MASUKGRIDLOCK_ATTHAKAWI_SB_1_0_LSXT        27702347#4_0
MASUKGRIDLOCK_ATTHAKAWI_SB_2_0_LXXX        27702347#6_0
MASUKGRIDLOCK_SUKHUMVUT22_SB_0_0_XSXX    -453669106#1_0
MASUKGRIDLOCK_SUKHUMVUT24_SB_0_0_XSXX     328942767#2_0
Name: id, Length: 425, dtype: object

In [5]:
ID.loc['820373198#0_3', 'name']

'RAMA4_WB_2_3_XSXX'

In [6]:
namedetector_csv = pd.read_csv('namedetector_KASEMRAT_flow.csv')
namedetector_df = pd.DataFrame(namedetector_csv, columns = ['name' , 'id'])
NAME_D = namedetector_df.set_index('name')
ID_D = namedetector_df.set_index('id')
listdetector = open("namedetector_KASEMRAT_flow.txt", "r")
detector = {}
for l in listdetector:
    l = l.strip().split(' ')
    if len(l)> 1:
        d = []
        for detec in l[2:]:
            if type(NAME_D.loc[detec,'id']) == str:
                d.append(NAME_D.loc[detec,'id'])
            else : d.append(NAME_D.loc[detec,'id'][0])
    if str(l[0])!= '':
        detector[str(l[0])] = d
list_detector= list(detector.keys())

In [7]:
NAME_D.head()

,id
name,
RAMA4_EB_1_4_XSXX,D459492917#0_4
KLONGTEI_EB_0_0_XSXX,D825786400_0
KLONGTEI_EB_0_1_XSXX,D825786400_1
KLONGTEI_EB_0_2_XSXX,D825786400_2
KLONGTEI_EB_0_3_XSXX,D825786400_3


In [8]:
['KASEMRAT_EB_FPX_TP2',50]

['KASEMRAT_EB_FPX_TP2', 50]

In [9]:
# type(NAME_D.loc['KLONGTEI_EB_0_0_XSXX','id'])
# for i in range(len(list(detector.keys()))):
#     print(i , list(detector.keys())[i])

In [10]:
def get_hot_encoding_current_phase():
    number_phase = [4,9,7,5,4,3]
    current_phase = [traci.trafficlight.getPhase(junction_id[key]) for key in junction_id.keys()]
#     current_phase = [0,2,1,1,1,1]
    hot_encoding_current_phase = np.array([])
    for i in range(len(current_phase)):
        binary_phase = np.zeros(number_phase[i])
        binary_phase[current_phase[i]] = 1
        hot_encoding_current_phase = np.concatenate((hot_encoding_current_phase, binary_phase), axis=None)
    return current_phase, hot_encoding_current_phase


In [11]:
# ID.loc['']#, '820373198#0', '820373196#0', '482209831#0']

In [12]:
def get_state_attention(current_phase, hot_encoding_current_phase):
#     current_phase = [KLONGTEI phase, RAMA4 phase,NARANONG phase, SUNLAKAKHON phase, KASEMRAT phase, ATTHAKAWI_RAMA4 phase]
    MAP_RAMA4 = {0: [['RAMA4_EB_FPX_TP1',92] , ['KASEMRAT_EB_FPX_TP2_RAMA4',80], ['NARANONG_SW_FPX_TP1',80]],
                 1: [['RAMA4_EB_FP1_TP3',10], ['RAMA4_NB_FPX_TP5',12.77]],
                 2: [['RAMA4_WB_FP2_TP4',10], ['RAMA4_EB_FPX_TP1',92], ['RAMA4_NB_FPX_TP5',12.77]],
                3: [['RAMA4_EB_FP3_TP2',40], ['RAMA4_WB_FP3_TP2',8.8], ['RAMA4_EB_FPX_TP1',92], ['RAMA4_NB_FPX_TP5',12.77 ]],
                4: [['RAMA4_SB_FP4_TP5',10], ['RAMA4_EB_FPX_TP1',92], ['RAMA4_NB_FPX_TP5',12.77 ]],
                5: [['RAMA4_NB_FP5_TP1',5], ['RAMA4_EB_FPX_TP1',92]],
                6: [['KASEMRAT_EB_FPX_TP2_RAMA4',80]],
                7: [['KASEMRAT_EB_FPX_TP2_RAMA4',80]],
                8: [['KASEMRAT_EB_FPX_TP2_RAMA4',80]]}
    MAP_KLONGTEI = {0: [['NARANONG_SB_FP5_TP6',80], ['RAMA4_EB_FP1_TP3',10]],
                    1: [['RAMA4_EB_FP3_TP2',40], ['RAMA4_EB_FP1_TP3',10]],
                    2: [['KLONGTEI_NB_FP3_TPX',10]],
                   3: [['RAMA4_EB_FP3_TP2',40], ['RAMA4_EB_FP1_TP3',10]] }
    MAP_NARANONG = {0: [['RAMA4_NB_FPX_TP5',12.77]],
                1: [['NARANONG_SB_FP1_TP2',30], ['NARANONG_EB_FP1_TP2',10], ['NARANONG_WB_FPX_TP2',83.17]],
                2: [['NARANONG_WB_FP2_TP3',5], ['NARANONG_WB_FP2_TP3_FLOW',70.83], ['NARANONG_SW_FPX_TP1',70.8], 
                   ['NARANONG_WB_FPX_TP2',83.17]],
                3: [['NARANONG_EB_FP3_TP4',10], ['NARANONG_SW_FPX_TP1',70.83], ['NARANONG_WB_FPX_TP2',83.17]],
                4: [['NARANONG_WB_FP4_TP5',5], ['NARANONG_SW_FPX_TP1',70.83], ['NARANONG_WB_FPX_TP2',83.17]],
                5: [['NARANONG_SB_FP5_TP6',5], ['NARANONG_SW_FPX_TP1',70.83],['NARANONG_WB_FPX_TP2',83.17]], 
                6: [['NARANONG_NB_FP6_TP1',5], ['NARANONG_SW_FPX_TP1',70.83], ['NARANONG_WB_FPX_TP2',83.17]] }
    MAP_SUNLAKAKHON = {0: [['NARANONG_WB_FPX_TP2',83.17]],
                    1: [['SUNLAKAKHON_SB_FP1_TPX',10]],
                    2: [['SUNLAKAKHON_NB_FP2_TP3',10], ['SUNLAKAKHON_SB_FPX_TP1',72.5]],
                    3:[['SUNLAKAKHON_EB_FP3_TP4',10], ['SUNLAKAKHON_WB_FP3_TP4',55.09], ['SUNLAKAKHON_SB_FPX_TP1',72.5]],
                    4:[['SUNLAKAKHON_SB_FP4_TP1',10], ['SUNLAKAKHON_SB_FPX_TP1',72.5]] }
    MAP_KASEMRAT = {0: [['SUNLAKAKHON_SB_FPX_TP1',80], ['MASUKGRIDLOCK_ARI_NB_FPX_TP2',20]],
                    1:[['KASEMRAT_EB_FPX_TP2',35.83], ['KASEMRAT_NB_FPX_TP3',90], ['MASUKGRIDLOCK_ARI_NB_FPX_TP2',20],\
                       ['MASUKGRIDLOCK_SUKHUMVUT_FPX_TP2',50]],
                    2:[['KASEMRAT_EB_FP2_TP1',20], ['KASEMRAT_NB_FPX_TP3',90]],
                    3:[['KASEMRAT_EB_FPX_TP2',35.83], ['MASUKGRIDLOCK_ARI_NB_FPX_TP2',20], ['MASUKGRIDLOCK_SUKHUMVUT_FPX_TP2',50]]}
    MAP_ATTHAKAWI_RAMA4 = {0:[['KASEMRAT_EB_FPX_TP2',50]],
                           1: [['MASUKGRIDLOCK_SUKHUMVUT_FPX_TP2',50], ['MASUKGRIDLOCK_ARI_NB_FPX_TP2', 20]],
                          2:[['KASEMRAT_EB_FPX_TP2_RAMA4',85]]}
    MAP = [MAP_KLONGTEI, MAP_RAMA4, MAP_NARANONG, MAP_SUNLAKAKHON,  MAP_KASEMRAT, MAP_ATTHAKAWI_RAMA4]
    state_attention = np.zeros(31)
    for i in range(len(current_phase)):
        for e in MAP[i][current_phase[i]]:
            occupancy = get_occupancy_average_percent(detector[e[0]]) 
            Index_detector = list_detector.index(e[0]) 
            if occupancy <= e[1]:
                state_attention[Index_detector] = 1
#     print(state_attention)
    state = np.concatenate((state_attention, hot_encoding_current_phase), axis=None)
    return state



In [13]:
#reset the environment
def start():
    sumoBinary = checkBinary('sumo')
    traci.start([sumoBinary, "-c", "KASEMRAT-SUMO-UsingBookNetFile/osm.sumocfg",
                             "--tripinfo-output", "tripinfo.xml", '--start','true','--quit-on-end','true','--time-to-teleport','-1',
                ])

In [14]:
def get_occupancy_average_percent(detector_id): 
    #get occupancy average for all detector in list of detector_id and scale by (Vehicle Length + MinimumGap)/MinimumGap 
    #Vehicle Length = 4.62 MinimumGap = 2.37
    occupancy = (sum([traci.lanearea.getLastStepOccupancy(e) for e in detector_id])/len(detector_id))*((4.62+2.37)/4.62)
    return occupancy

In [15]:
def get_throughput():
    loopID = traci.inductionloop.getIDList()
    throughput = sum([traci.inductionloop.getLastStepVehicleNumber(i) for i in loopID if traci.inductionloop.getLastStepMeanSpeed(i) > 0])
    return throughput

In [16]:
def get_backlog():
#     laneID = traci.lane.getIDList()
    backlog = sum([traci.lanearea.getLastStepVehicleNumber(i) for i in NAME.loc[:,'id']])
    return backlog

In [17]:
def get_reward():
    throughput = 0
    for i in range(5):
        traci.simulationStep()
        throughput += get_throughput()
    
    backlog = get_backlog()
    reward = throughput - 0.004*backlog
    return reward, throughput, backlog

In [18]:
number_phase = [4,9,7,5,4,3]
def set_current_phase(action, current_phase):
    if action < 4:
        phase = action
        current_phase[0] = phase
    elif action < 13:
        phase = (action-4)
        current_phase[1] = phase
    elif action < 20:
        phase = (action-13)
        current_phase[2] = phase
    elif action < 25:
        phase = (action-20)
        current_phase[3] = phase
    elif action < 29:
        phase = (action-25)
        current_phase[4] = phase
    else:
        phase = (action-29)
        current_phase[5] = phase
    for i in range (6):
        traci.trafficlight.setPhase(junction_id[junction_name[i]], current_phase[i])
    if current_phase[5] == 2:
        traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_1_0_LSXT', 'id'], ['passenger'])
        traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_2_0_LXXX', 'id'], ['passenger'])
        traci.lane.setAllowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_0_0_XSRT', 'id'], ['passenger'])
    elif current_phase[5] == 1:
        traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_1_0_LSXT', 'id'], ['passenger'])
        traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_2_0_LXXX', 'id'], ['passenger'])
        traci.lane.setDisallowed(NAME.loc['MASUKGRIDLOCK_ATTHAKAWI_SB_0_0_XSRT', 'id'], ['passenger'])
    return 

In [19]:
reward_memory = []
def plot_durations():
    print('show')
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(reward_memory, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Step')
    plt.ylabel('Reward')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated

In [20]:
class MyEnv3(gym.Env):
    def __init__(self, env_config):
        self.count = 0
        self.reward = 0
        self.rewards = 0
        self.throughputs = 0
        self.backlogs = 0
        self.current_phase = [1,1,1,1,1,1]
        self.done = False
        self.reward_memory = []
        self.action_space = gym.spaces.Discrete(31)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(62,), dtype=np.int8)
        dateTimeObj = datetime.datetime.now()
        self.dateTimeObj = dateTimeObj.strftime("%d-%b-%Y-%H-%M-%S")
        print(self.dateTimeObj)
        with open( "./Raytest/ray_results/"+self.dateTimeObj+".csv" , 'a', newline='') as csv_file:
                header = ['rewards', 'throughput','backlog']
                writer = csv.DictWriter(csv_file, fieldnames = header)
                writer.writeheader()
        
    def reset(self):
        
        self.reward_memory.append(self.rewards)
        self.count = 0
        start()
        self.reward = 0
        self.rewards = 0
        self.throughputs = 0
        self.backlogs = 0
        print(self.reward_memory)
        current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
        state = get_state_attention(current_phase, hot_encoding_current_phase)
#         print(state)
        self.done = False
        self.current_phase = current_phase
        if len(self.reward_memory)%100 == 0:
            print('memory',self.reward_memory[-10:])
        return state 
    
    
    def step(self, action):
        set_current_phase(action, self.current_phase)
#         print(action)
        current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
        self.reward, throughput, backlog= get_reward()
        self.rewards += self.reward
        self.throughputs += throughput
        self.backlogs += backlog
        state = get_state_attention(current_phase, hot_encoding_current_phase)
        if np.isnan(self.reward) == True:
            print('HELP', type(self.reward))
        self.count += 1
        self.current_phase = current_phase
#         print('count', self.count)
        self.done = False
        if self.count >= 2880: #2880
            traci.close()
            self.done = True
            with open( "Raytest/ray_results/"+self.dateTimeObj+".csv" , 'a', newline='') as csv_file:
                header = ['rewards', 'throughput','backlog']
                writer = csv.DictWriter(csv_file, fieldnames = header)
                writer.writerow({'rewards': self.rewards, 
                                 'throughput': self.throughputs,
                                'backlog': self.backlogs})

        
        return_state = np.array(state).astype(np.int8)
#         print(return_state)
        info = {"throughput": throughput,
                "backlog":backlog
                }
        info = {**info}
        print(info)
        return return_state , self.reward, self.done, info

In [ ]:
if __name__ == "__main__":
    print("asd")
    env = MyEnv3(env_config=None)

    for i_episode in range(1):
        observation = env.reset()
        for t in range(3000):
            # env.render()
#             print(observation)
            action = env.action_space.sample()
            # print('action' ,action)
            observation, reward, done, info = env.step(action)
            if done:
                print("Episode finished after {} timesteps")
                break
    env.close()

asd
06-Apr-2021-21-32-11
[0]
{'throughput': 0, 'backlog': 4}
{'throughput': 0, 'backlog': 7}
{'throughput': 0, 'backlog': 11}
{'throughput': 1, 'backlog': 16}
{'throughput': 1, 'backlog': 21}
{'throughput': 0, 'backlog': 30}
{'throughput': 0, 'backlog': 30}
{'throughput': 1, 'backlog': 41}
{'throughput': 0, 'backlog': 45}
{'throughput': 1, 'backlog': 52}
{'throughput': 2, 'backlog': 65}
{'throughput': 0, 'backlog': 71}
{'throughput': 2, 'backlog': 81}
{'throughput': 1, 'backlog': 88}
{'throughput': 9, 'backlog': 93}
{'throughput': 8, 'backlog': 100}
{'throughput': 5, 'backlog': 112}
{'throughput': 7, 'backlog': 116}
{'throughput': 3, 'backlog': 126}
{'throughput': 3, 'backlog': 145}
{'throughput': 4, 'backlog': 142}
{'throughput': 8, 'backlog': 150}
{'throughput': 7, 'backlog': 165}
{'throughput': 2, 'backlog': 163}
{'throughput': 5, 'backlog': 181}
{'throughput': 8, 'backlog': 184}
{'throughput': 5, 'backlog': 185}
{'throughput': 0, 'backlog': 194}
{'throughput': 3, 'backlog': 204}
{'

In [ ]:
# import ray
# import ray.tune  as tune
# from ray.tune import Callback
# from ray.rllib.utils import merge_dicts
# from ray.rllib.agents.dqn.dqn import calculate_rr_weights, \
#     DEFAULT_CONFIG as DQN_CONFIG, DQNTrainer, validate_config
# # env = gym.make('CartPole-v0')


# class MyCallback(Callback):
#     def on_trial_start(self, iteration, trials, trial, **info):
#         print(f"I am in callback. This is iteration {iteration} inside trial {trial}")
# #         dateTimeObj = datetime.datetime.now()
# #         dateTimeObj = dateTimeObj.strftime("%d-%b-%Y-%H-%M-%S-%f")
# #         with open( "./Raytest/ray_results/"+dateTimeObj+".csv" , 'a', newline='') as csv_file:
# #                 header = ['rewards', 'throughput','backlog']
# #                 writer = csv.DictWriter(csv_file, fieldnames = header)
# #                 writer.writeheader()
# #         print(info)
#     def on_trial_result(self, iteration, trials, trial, result, **info):
#         print("I am in second callback. Got result:", info)
# #         with open( "./Raytest/ray_results/"+str(trial)+".csv" , 'a', newline='') as csv_file:
# #             header = ['rewards']
# #             writer = csv.DictWriter(csv_file, fieldnames = header)
# #             writer.writeheader()
# #             writer.writerow({'rewards': result["episode_reward_mean"]})
            

# def trial_name_id(trial):
#     return f"{trial.trainable_name}_{trial.trial_id}"

# if __name__ == "__main__":
#     ray.shutdown()
#     ray.init(
# #              object_store_memory=int(1e9),  # 4gb
# #              redis_max_memory=int(1e9)  #2gb
#              )
#     experiment_spec = tune.Experiment(
#         trial_name_creator=trial_name_id,
#         name = "experiment_apex6",
#         run = "APEX",
#         local_dir = "./Raytest/ray_results",
#         checkpoint_freq = 3,
#         checkpoint_at_end = True,
#         log_to_file=True,
#         config = {
# #         "optimizer": merge_dicts(
# #             DQN_CONFIG["optimizer"], {
# #                 "max_weight_sync_delay": 400,
# #                 "num_replay_buffer_shards": 4,
# #                 "debug": False
# #             }),
# #         "n_step": 3,
        
#             "adam_epsilon": 1e-8,
#             # If not None, clip gradients during optimization at this value
            
#         "num_gpus": 0,
#         "dueling": False,
#         "double_q": False,
#         "num_workers": 15,
#         "buffer_size": 100000,
#         "framework": "tf",
#         "learning_starts": 28800, #2160
#         "train_batch_size": 540,
# #             "num_samples": 20,
#         "rollout_fragment_length": 50,
#         "target_network_update_freq": 540,
#         "prioritized_replay": True,
#         "timesteps_per_iteration": 2880, #2880
# #         "exploration_config": {"type": "PerWorkerEpsilonGreedy"},
# #         "worker_side_prioritization": True,
# #         "min_iter_time_s": 30,
#         # If set, this will fix the ratio of replayed from a buffer and learned
#         # on timesteps to sampled from an environment and stored in the replay
#         # buffer timesteps. Otherwise, replay will proceed as fast as possible.
# #         "training_intensity": None,
#          "worker_side_prioritization": False,
#          "lr": 5e-4,
#             "gamma": 0.6,

            
#             "env": MyEnv3 ,

            
#             }
        
#         )
#     results = tune.run_experiments(experiment_spec, 
#                                   callbacks=[MyCallback()])

In [ ]:
# reward_memory = []
# def plot_durations():
#     print('show')
#     plt.figure(2)
#     plt.clf()
#     durations_t = torch.tensor(reward_memory, dtype=torch.float)
#     plt.title('Training...')
#     plt.xlabel('Step')
#     plt.ylabel('Reward')
#     plt.plot(durations_t.numpy())
#     # Take 100 episode averages and plot them too
#     if len(durations_t) >= 100:
#         means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
#         means = torch.cat((torch.zeros(99), means))
#         plt.plot(means.numpy())
#     plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
# env = MyEnv3(env_config=None)
# action_space_size = env.action_space.n
# state_space_size = 2^57
# columns_name = ['state']+list(range(0, 30))
# q_table = pd.DataFrame(columns = columns_name).set_index('state')

# print(q_table)

In [ ]:
# traci.close()

In [ ]:
# num_episodes = 1000
# max_steps_per_episode = 2880 # but it won't go higher than 1

# learning_rate = 0.1
# discount_rate = 0.99

# exploration_rate = 1
# max_exploration_rate = 1
# min_exploration_rate = 0.01

# exploration_decay_rate = 0.01 #if we decrease it, will learn slower

In [ ]:
# traci.close()

In [ ]:
# rewards_all_episodes = []

# # Q-Learning algorithm
# for episode in range(num_episodes):
#     state = env.reset()
#     state=''.join(list(str(s) for s in state))
#     print(state)
#     done = False
#     rewards_current_episode = 0
    
#     for step in range(max_steps_per_episode):
#         if state not in q_table.index:
#             q_table.loc[state,:] = 0
#         # Exploration -exploitation trade-off
#         exploration_rate_threshold = random.uniform(0,1)
#         if exploration_rate_threshold > exploration_rate: 
#             action = (q_table[state,:].idxmax())
#         else:
#             action = env.action_space.sample()
            
#         new_state, reward, done, info = env.step(action)
#         new_state=''.join(list(str(s) for s in new_state))
#         # Update Q-table for Q(s,a)
#         if new_state not in q_table.index:
#             q_table.loc[new_state,:] = 0
#         q_table.loc[str(state), action] = (1 - learning_rate) * q_table.loc[str(state), action] + \
#             learning_rate * (reward + discount_rate * (q_table.loc[new_state,:].max()))
            
#         state = new_state
#         rewards_current_episode += reward
        
#         if done == True: 
#             break
            
#     # Exploration rate decay
#     exploration_rate = min_exploration_rate + \
#         (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
    
#     rewards_all_episodes.append(rewards_current_episode)
    
#     if episode%1 == 0:
#         plt.figure(2)
#         plt.clf() 
#         line_up = plt.plot(rewards_all_episodes)
#         plt.legend([line_up], ['reward'])
#         plt.pause(0.001)
    
# # Calculate and print the average reward per 10 episodes
# rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes / 100)
# count = 100
# print("********** Average  reward per thousand episodes **********\n")

# for r in rewards_per_thousand_episodes:
#     print(count, ": ", str(sum(r / 100)))
#     count += 100
    
# # Print updated Q-table
# print("\n\n********** Q-table **********\n")
# print(q_table)

In [ ]:
# if __name__ == "__main__":
#     print("asd")
#     env = MyEnv3(env_config=None)

#     for i_episode in range(1):
#         observation = env.reset()
#         for t in range(1000):
#             # env.render()
# #             print(observation)
#             action = env.action_space.sample()
# #             print('action' ,action)
#             observation, reward, done, info = env.step(action)
#             if done:
#                 print("Episode finished after {} timesteps")
#                 break
#     env.close()


In [ ]:
# traci.close()

In [ ]:
# import ray
# import ray.tune  as tune
# from ray.tune import Callback
# from ray.rllib.utils import merge_dicts
# from ray.rllib.agents.dqn.dqn import calculate_rr_weights, \
#     DEFAULT_CONFIG as DQN_CONFIG, DQNTrainer, validate_config
# # env = gym.make('CartPole-v0')


# class MyCallback(Callback):
#     def on_trial_start(self, iteration, trials, trial, **info):
#         print(f"I am in callback. This is iteration {iteration} inside trial {trial}")
#         print(info)
#     def on_trial_result(self, iteration, trials, trial, result, **info):
#         print("I am in second callback. Got result:", info, result)
#         with open( "/Raytest/ray_results/"+str(trial)+".csv" , 'a', newline='') as csv_file:
#             header = ['rewards']
#             writer = csv.DictWriter(csv_file, fieldnames = header)
#             writer.writeheader()
#             writer.writerow({'rewards': result["episode_reward_mean"]})
            

# def trial_name_id(trial):
#     return f"{trial.trainable_name}_{trial.trial_id}"

# if __name__ == "__main__":
#     ray.shutdown()
#     ray.init(
# #              object_store_memory=int(1e9),  # 4gb
# #              redis_max_memory=int(1e9)  #2gb
#              )
#     experiment_spec = tune.Experiment(
#         trial_name_creator=trial_name_id,
#         name = "experiment_apex5",
#         run = "APEX",
#         local_dir = "/Raytest/ray_results",
#         checkpoint_freq = 3,
#         checkpoint_at_end = True,
#         log_to_file=True,
#         config = {
# #         "optimizer": merge_dicts(
# #             DQN_CONFIG["optimizer"], {
# #                 "max_weight_sync_delay": 400,
# #                 "num_replay_buffer_shards": 4,
# #                 "debug": False
# #             }),
# #         "n_step": 3,
#         "num_gpus": 0,
#         "dueling": False,
#         "double_q": False,
#         "num_workers": 4,
#         "buffer_size": 100000,
#         "framework": "tf",
#         "learning_starts": 2160, #2160
#         "train_batch_size": 540,
# #             "num_samples": 20,
#         "rollout_fragment_length": 50,
#         "target_network_update_freq": 540,
#         "prioritized_replay": True,
#         "timesteps_per_iteration": 2880,
# #         "exploration_config": {"type": "PerWorkerEpsilonGreedy"},
# #         "worker_side_prioritization": True,
# #         "min_iter_time_s": 30,
#         # If set, this will fix the ratio of replayed from a buffer and learned
#         # on timesteps to sampled from an environment and stored in the replay
#         # buffer timesteps. Otherwise, replay will proceed as fast as possible.
# #         "training_intensity": None,
#          "worker_side_prioritization": False,
 
#             "gamma": 0.7,

            
#             "env": MyEnv3 ,

            
#             }
        
#         )
#     results = tune.run_experiments(experiment_spec, 
#                                   callbacks=[MyCallback()])


In [ ]:
x = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 0.])

In [ ]:
len(x)